# Forensics - Create Windows VM

__Notebook Version:__ 1.0<br>
__Python Version:__ Python 3.6 (including Python 3.6 - AzureML)<br>
__Required Packages:__ Azure 4.0.0<br>
__OS Supported:__<br>
    -  Windows
__Platforms Supported:__<br>
    -  Azure Notebooks Free Compute
    -  Azure Notebooks DSVM
__Data Source Required:__<br>
    -  no
    
### Description
The notebook provides sample code to create a Windows VM.

<font color=red>When you switch between Azure Notebooks Free Compute and Data Science Virtual Machine (DSVM), you may need to select Python version: please select Python 3.6 for Free Compute, and Python 3.6 - AzureML for DSVM.</font>

## Prerequisite check

In [ ]:
# only run once, current version 0.1.2
!pip install --upgrade Sentinel-Utilities

In [ ]:
import SentinelUtils
# checking Python version
check = SentinelUtils.version_management.ModuleVersionCheck()
py_check = check.validate_python('3.6.0')
if py_check.requirement_met == False:
    print('Please select Python 3.6 or Python 3.6 - AzureML at the upper right corner')
else:
    print('Please continue')

In [ ]:
# checking required packages
mods_check = check.validate_installed_modules(['Azure>=4.0.0'])
for mod_info in mods_check:
    if mod_info.requirement_met == False:
        print('Please install {} {} at the following cell.'.format(mod_info.name, mod_info.required_version))

In [ ]:
# Please install required packages based on the check at last cell
#!pip install azure

## Table of Contents

1. Retrieve Azure Resource Information
2. Authentication to Azure Resource Management 
3. Create Windows VM

## 1. Retrieve Azure Resource Information

In [ ]:
path = %env PATH
dsvm = False
if '/dsvm/' in path:
    dsvm = True

# information from config.json
if dsvm == False:
    # Run this if you are using Free Compute
    tenant_id = SentinelUtils.config_reader.ConfigReader.read_config_values("config.json")[0]
    subscription_id = SentinelUtils.config_reader.ConfigReader.read_config_values("config.json")[1]
    resource_group = SentinelUtils.config_reader.ConfigReader.read_config_values("config.json")[2]
else:
    # Run this if you are using DSVM.  You need to copy the values from config.json, if the file has no value, then you need to go to Log Analytics Portal to get the information.
    tenant_id = input('tenant_id:')
    subscription_id = input('subscription_id:')
    resource_group = input('resource_group:')

## 2. Authentication to Azure Resource Management

In [ ]:
from azure.common.credentials import ServicePrincipalCredentials
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.compute import ComputeManagementClient
from azure.mgmt.network import NetworkManagementClient
from azure.mgmt.compute.models import DiskCreateOption

1. Select Authentication Methods<br>
If multi-factors authentication is enabled for your tenant, you must use Service Principal to authentication to Azure resource management.  Otherwise, you may choose either Service Principal or User ID/Password.

In [ ]:
import ipywidgets as widgets
from IPython.display import display
auth_method = ['Service Principal', 'User ID/Password']
selected_auth = widgets.Dropdown(options=auth_method, value=auth_method[0],description='Auth Method:')
display(selected_auth)

2. Authentication

In [ ]:
if selected_auth.value == 'Service Principal':
    credentials = ServicePrincipalCredentials(
        client_id=input('client_id:'), 
        secret=input('secret:'), 
        tenant=tenant_id)
else:
    from azure.common.credentials import UserPassCredentials
    import getpass
    uid = input('User ID:')
    pwd = getpass.getpass()
    credentials = UserPassCredentials(uid, pwd)
    
client = ResourceManagementClient(credentials, subscription_id)

## 3. Create Windows VM

In [ ]:
resource_group_client = ResourceManagementClient(credentials, subscription_id)
compute_client = ComputeManagementClient(credentials, subscription_id)
network_client = NetworkManagementClient(credentials, subscription_id)

In [ ]:
# User input
vm_name = input('vm_name:')
vm_location = input('vm_location:')
group_name = input('resource_group_name:')

In [ ]:
# functions for creating VM
def create_resource_group(resource_group_client):
    resource_group_params = { 'location': vm_location }
    resource_group_result = resource_group_client.resource_groups.create_or_update(
        group_name, 
        resource_group_params
    )
    
def create_availability_set(compute_client):
    avset_params = {
        'location': vm_location,
        'sku': { 'name': 'Aligned' },
        'platform_fault_domain_count': 3
    }
    availability_set_result = compute_client.availability_sets.create_or_update(
        group_name,
        'myAVSet',
        avset_params
    )
    
def create_public_ip_address(network_client):
    public_ip_addess_params = {
        'location': vm_location,
        'public_ip_allocation_method': 'Dynamic'
    }
    creation_result = network_client.public_ip_addresses.create_or_update(
        group_name,
        'myIPAddress',
        public_ip_addess_params
    )

    return creation_result.result()

def create_vnet(network_client):
    vnet_params = {
        'location': vm_location,
        'address_space': {
            'address_prefixes': ['10.0.0.0/16']
        }
    }
    creation_result = network_client.virtual_networks.create_or_update(
        group_name,
        'myVNet',
        vnet_params
    )
    return creation_result.result()

def create_subnet(network_client):
    subnet_params = {
        'address_prefix': '10.0.0.0/24'
    }
    creation_result = network_client.subnets.create_or_update(
        group_name,
        'myVNet',
        'mySubnet',
        subnet_params
    )

    return creation_result.result()

def create_nic(network_client):
    subnet_info = network_client.subnets.get(
        group_name, 
        'myVNet', 
        'mySubnet'
    )
    publicIPAddress = network_client.public_ip_addresses.get(
        group_name,
        'myIPAddress'
    )
    nic_params = {
        'location': vm_location,
        'ip_configurations': [{
            'name': 'myIPConfig',
            'public_ip_address': publicIPAddress,
            'subnet': {
                'id': subnet_info.id
            }
        }]
    }
    creation_result = network_client.network_interfaces.create_or_update(
        group_name,
        'myNic',
        nic_params
    )

    return creation_result.result()

In [ ]:
def create_vm():
    import azure

    nic = network_client.network_interfaces.get(
            group_name, 
            'myNic'
        )

    result = compute_client.virtual_machines.create_or_update(
        group_name,
        vm_name,
        azure.mgmt.compute.models.VirtualMachine(
            location = vm_location,
            os_profile = azure.mgmt.compute.models.OSProfile(
                admin_username = 'xxxx',
                admin_password = 'xxxx',
                computer_name = vm_name,
            ),
            hardware_profile = azure.mgmt.compute.models.HardwareProfile(
                vm_size = azure.mgmt.compute.models.VirtualMachineSizeTypes.standard_b2s
            ),
            network_profile=azure.mgmt.compute.models.NetworkProfile(
                network_interfaces=[
                    azure.mgmt.compute.models.NetworkInterfaceReference(
                        id=nic.id,
                        primary=True
                    ),
                ],
            ),
            storage_profile=azure.mgmt.compute.models.StorageProfile(
                os_disk=azure.mgmt.compute.models.OSDisk(
                    caching=azure.mgmt.compute.models.CachingTypes.none,
                    create_option=azure.mgmt.compute.models.DiskCreateOptionTypes.from_image,
                    name='vhdsnapshot',
                ),
                image_reference = azure.mgmt.compute.models.ImageReference(
                    publisher='MicrosoftWindowsServer',
                    offer='WindowsServer',
                    sku='2012-R2-Datacenter',
                    version='latest',
                ),
            ),
        ),
    )

In [ ]:
# create resource group
async_resource_group_create = create_resource_group(resource_group_client)

In [ ]:
# create availability set
create_availability_set(compute_client)

In [ ]:
# create public IP address
creation_result = create_public_ip_address(network_client)
print(creation_result)

In [ ]:
# create virtual network
creation_result = create_vnet(network_client)
print(creation_result)

In [ ]:
# create a subnet
creation_result = create_subnet(network_client)
print(creation_result)

In [ ]:
# create ntework interface client
creation_result = create_nic(network_client)
print(creation_result)

In [ ]:
# create VM
creation_result = create_vm()
print(creation_result)